## Patents term counts #
#### make requests and get all results by keyword from patentsview api
* Not necessarily better than just downloading and parsing the text?
* Note - unsure how texti s cleaned if at all , and there are less results than when I do a google patents search 


* note that api returns X results maximum, so need to go through them all
https://patentsview.org/apis/api-query-language#sort_parameter



TODO: check that my replacing of spaces by `%20` is legit and has same results



https://stackoverflow.com/questions/6386308/http-requests-and-json-parsing-in-python
https://stackoverflow.com/questions/60464347/downloading-data-from-multiple-pages-website-with-requests-method-in-python

#### get all pages/results - 
https://stackoverflow.com/questions/63089935/python-requests-get-json-data-with-pagination
```
url = 'secret_url?page={}'
page_number = 1
data = requests.get(url.format(page_number), headers=headers).json()
total_pages = data['pages']['pages']
for page_number in range(page, total_pages + 1):
    data = requests.get(url.format(page_number), headers=headers).json()
```



In [1]:
import requests
import pandas as pd
import re
import random
import time

from pandas.io.json import json_normalize
# url='http://json-homework.task-sss.krasilnikov.spb.ru/api/groups/getmembers?api_key=9f66a575a6cfaaf7e43177317461d057&group_id=4508123&page=1'
# data=rq.get(url)
# data1=json.loads(data.text)
# data1=json_normalize(json.loads(data.text)["response"])


## sleep between scrapes/reqs

# time.sleep(random.randint(1,7))

SAVE = True

https://api.patentsview.org/patents/query?q={"_and": [{"_gte":{"patent_date":"2001-01-01"}},{"_or": [{"_text_all":{"patent_abstract":"international"}},{"_text_all":{"patent_title":"pizza"}}]}]}&f=["patent_number", "patent_date"]&o={"per_page":10000}&f=["patent_title","patent_abstract","patent_number", "patent_kind", "patent_date"]


### terms

* Need also additional terms
* mesh terms ? 
from 

#### TODO: add mesh terms scraping

In [2]:
## copied list from trends pubmed scrape notebooks - need to add more

KW_LIST = list(set([
    "miRNA","prion","rnase","ncRNA","lncRNA","PCR","mass spectrometry",
    "ancient dna", "carbon dating", ## ancient dna - methods changed over time, C14.. 
    "sleep","coral","SNP","GWAS",
    "neural networks","artificial neural networks","machine learning","genetic algorithm",
    "antibiotic","antibiotic resistance","viral therapy","gene therapy",
    "microglia","junk dna",
    "eugenics","genetic engineering","vitamin D","vitamin C","race","ethnic differences",
    "influenza", "archaea","stem cells", "vaccine","Epidemiology",
    "Synthetic Biology","epigenetics","perceptron","eugenics","lithium","lipid",
    "Mononucleosis","epstein-barr","multiple sclerosis","autoimmune",
    "ribosome","ribozyme",
    "zebra fish","mouse","rnai","CRISPR","DNA array","RNA","NGS","single cell",
    "illumina","nanopore","BRCA1","BRCA2","cholesterol", "soy milk",
    "cumin","biosimilar","homeopathy","natural medicine","single cell","FACS",
    "Drug repurposing","LSD","cannabis","cannabidiol","MRI","fMRI","EEG","BLAST","HMM", ## profile-profile, homology,   
        "connectome","metabolome","metagenomics",
    "autism", "savant", "autoimmune" , "inflammatory",
    ### https://en.wikipedia.org/wiki/Neurotransmitter
    "glutamate", "GABA", "acetylcholine", "glycine" , "serotonin", "epinephrine", "Norepinephrine",
    "dopamine", "substance P", "opioids", "cocaine", "Histamine", "neuropeptide", "neutrotransmitter",
    ## some brain regions: + https://en.wikipedia.org/wiki/List_of_regions_in_the_human_brain
    "amygdala", "hippocampus", "neocortex", "basal ganglia", "lateral ventricles",
    "thalamus", "hypothalamus",
    "subthalamus", "pituitary gland","pineal gland" ,"cerebellum", "Medulla oblongata",
    "paleocortex" , "neocortex", "Brodmann area", "Prefrontal cortex", "Orbitofrontal cortex",
    "Dorsolateral prefrontal cortex", "somatosensory cortex", "Insular cortex", "Cingulate cortex",
    ### combine cannabis, canabidiol =  marijuana .  lsd.  
    ## psychiatric drugs, treatment 
    
#### mesh cats: 
    'Amino Acids, Peptides, and Proteins',
 'Anesthesia and Analgesia',
 'Animal Diseases',
 'Animal Structures',
 'Archaea',
 'Bacteria',
 'Bacterial Infections and Mycoses',
 'Behavior and Behavior Mechanisms',
 'Mental Disorders',
 'Biological Factors',
 'Biological Phenomena',
 'Biomedical and Dental Materials',
 'Body Regions',
 'Carbohydrates',
 'Cardiovascular Diseases',
 'Cardiovascular System',
 'Cell Physiological Phenomena',
 'Cells',
 'Circulatory and Respiratory Physiological Phenomena',
 'Dentistry',
 'Diagnosis',
 'Digestive System',
 'Digestive System Diseases',
 'Behavioral Disciplines and Activities',
 'Pharmaceutical Preparations',
 'Health Care Economics and Organizations',
 'Education',
 'Embryonic Structures',
 'Endocrine System Diseases',
 'Endocrine System',
 'Environment and Public Health',
 'Equipment and Supplies',
 'Eye Diseases',
 'Health Care Facilities, Manpower, and Services',
 'Female Urogenital Diseases and Pregnancy Complications',
 'Fluids and Secretions',
 'Geographic Locations',
 'Health Occupations',
 'Health Services Administration',
 'Hemic and Immune Systems',
 'Hemic and Lymphatic Diseases',
 'Heterocyclic Compounds',
 'Hormones, Hormone Substitutes, and Hormone Antagonists',
 'Human Activities',
 'Humanities',
 'Immune System Diseases',
 'Information Science',
 'Disorders of Environmental Origin',
 'Inorganic Chemicals',
 'Lipids',
 'Microbiological Phenomena',
 'Investigative Techniques',
 'Stomatognathic Diseases',
 'Musculoskeletal Diseases',
 'Musculoskeletal System',
 'Persons',
 'Congenital, Hereditary, and Neonatal Diseases and Abnormalities',
 'Neoplasms',
 'Nervous System',
 'Nervous System Diseases',
 'Nucleic Acids, Nucleotides, and Nucleosides',
 'Nutritional and Metabolic Diseases',
 'Occupational Diseases',
 'Ocular Physiological Phenomena',
 'Organic Chemicals',
 'Otorhinolaryngologic Diseases',
 'Parasitic Diseases',
 'Natural Science Disciplines',
 'Physiological Phenomena',
 'Polycyclic Compounds',
 'Population Characteristics',
 'Psychological Phenomena and Processes',
 'Reproductive and Urinary Physiological Phenomena',
 'Respiratory System',
 'Respiratory Tract Diseases',
 'Sense Organs',
 'Social Sciences',
 'Stomatognathic System',
 'Surgical Procedures, Operative',
 'Pathological Conditions, Signs and Symptoms',
 'Technology, Industry, and Agriculture',
 'Therapeutics',
 'Tissues',
 'Urogenital System',
 'Virus Diseases',
 'Viruses',
 'Wounds and Injuries',
 'Skin and Connective Tissue Diseases',
 'Health Care Quality, Access, and Evaluation',
 'Plant Structures',
 'Plant Physiological Phenomena',
 'Food and Beverages',
 'Chemical Actions and Uses',
 'Integumentary System',
 'Complex Mixtures',
 'Enzymes and Coenzymes',
 'Macromolecular Substances',
 'Publication Formats',
 'Publication Components',
 'Study Characteristics',
 'Support of Research',
 'Male Urogenital Diseases',
 'Physical Phenomena',
 'Chemical Phenomena',
 'Genetic Phenomena',
 'Immune System Phenomena',
 'Mathematical Concepts',
 'Musculoskeletal and Neural Physiological Phenomena',
 'Digestive System and Oral Physiological Phenomena',
 'Metabolic Phenomena',
 'Integumentary System Physiological Phenomena',
 'Viral Structures',
 'Bacterial Structures',
 'Fungal Structures',
 'Eukaryota',
 'Organism Forms',
 'Chemically-Induced Disorders',
    
        ## adding - NOT in origianl terms from pubmed :    
  "carbon nanotubes","crispr cas-9", "hydrophonics","aquaphonic agriculture",
"ancient bacteria", "ancient viruses",
"covid","synthetic biology", "buckyballs","ultrasound", "language model", "crypto","blockchain",
    "graph neural network",
]))


# KW_LIST = ["carbon nanotubes","crispr cas-9"]
# KW_LIST = ['Amino Acids, Peptides, and Proteins','Chemically-Induced Disorders']

In [3]:
res_all = {}#[] ## will hold results over all terms
res_term = [] ## will store list of dicts - results

failed_terms = []

In [4]:
### https://stackoverflow.com/questions/42521230/how-to-escape-curly-brackets-in-f-strings
# url = '''https://api.patentsview.org/patents/query?q={"_and": [{"_gte":{"patent_date":"2017-01-01"}},
#                                                             {"_or": [{"_text_all":{"patent_abstract":"international"}},{"_text_all":{"patent_title":"pizza"}}]}]}&f=["patent_number",
#                                                             "patent_date"]&o={"per_page":200,}'''

## as above with f string, double curly brackets:
# url = f'''https://api.patentsview.org/patents/query?q={{"_and": [{{"_gte":{{"patent_date":"2017-01-01"}}}},
# {{"_or": [{{"_text_all":{{"patent_abstract":"international"}}}},
# {{"_text_all":{{"patent_title":"pizza"}}}}]}}]}}&f=["patent_date"]&o={{"per_page":200,"page":1}}'''

In [5]:

# for kw_name in KW_LIST:
# #     kw_name = ''.join(filter(str.isalnum, kw_name))

#     kw_name =  re.sub(r'\W+', ' ', kw_name).lower().replace(" ","%20")
#     print(kw_name)

In [6]:
for kw_name in KW_LIST:
    print(kw_name)
    res_term = []
    time.sleep(random.randint(1,6))
    page_num =1 
#     kw = kw_name.lower().replace(" ","%20")#.replace(" ","+")#.replace(" ","%20") ## replace spaces
    kw =  re.sub(r'\W+', ' ', kw_name).lower().replace(" ","%20") ## replace spaces and puncts/-
    
    url = f'''https://api.patentsview.org/patents/query?q={{"_and": [{{"_gte":{{"patent_date":"1950-01-01"}}}},\
{{"_or": [{{"_text_all":{{"patent_abstract":"{kw}"}}}},\
{{"_text_all":{{"patent_title":"{kw}"}}}}]}}]}}&f=["patent_date"]&o={{"per_page":10000,"page":{page_num}}}'''

#     print(url)
    try:
        data = requests.get(url,headers={'User-Agent':"ddofer"}).json() ## , headers=headers
        data
        dates = [d.get('patent_date') for d in data["patents"]]
        res_term.extend(dates)

        while data["total_patent_count"]> len(res_term):
            page_num +=1

            url = f'''https://api.patentsview.org/patents/query?q={{"_and": [{{"_gte":{{"patent_date":"1950-01-01"}}}},\
    {{"_or": [{{"_text_all":{{"patent_abstract":"{kw}"}}}},\
    {{"_text_all":{{"patent_title":"{kw}"}}}}]}}]}}&f=["patent_date"]&o={{"per_page":10000,"page":{page_num}}}'''

            data = requests.get(url,headers={'User-Agent':f"ddofer@gmail_{page_num}"}).json()
    #         res_term.extend(data["patents"])
            dates = [d.get('patent_date') for d in data["patents"]] ## list of all dates
            res_term.extend(dates)

            ## sleep between scrapes/reqs
            time.sleep(random.randint(1,2))
        print(len(res_term))
        res_all[kw_name.replace("%20"," ")] = res_term
    except:
        print("-----------------------------\n BAD error:",kw)
        failed_terms.append(kw_name)
#         print(data)
#         print(url)

glutamate
1283
buckyballs
15
mass spectrometry
3525
illumina
4
metagenomics
17
Male Urogenital Diseases
-----------------------------
 BAD error: male%20urogenital%20diseases
ncRNA
11
neural networks
2635
neuropeptide
268
hippocampus
121
Genetic Phenomena
11
epinephrine
159
Mononucleosis
24
Technology, Industry, and Agriculture
3
Biological Factors
317
Eye Diseases
751
Geographic Locations
1809
Nervous System Diseases
1233
Mathematical Concepts
54
synthetic biology
58
archaea
36
Bacterial Infections and Mycoses
2
serotonin
1421
Natural Science Disciplines
-----------------------------
 BAD error: natural%20science%20disciplines
Biomedical and Dental Materials
7
ancient bacteria
2
MRI
6131
Circulatory and Respiratory Physiological Phenomena
-----------------------------
 BAD error: circulatory%20and%20respiratory%20physiological%20phenomena
Immune System Phenomena
5
blockchain
3319
Female Urogenital Diseases and Pregnancy Complications
-----------------------------
 BAD error: female%20

In [7]:
dates

['2018-06-19',
 '2018-06-19',
 '2018-06-19',
 '2018-06-19',
 '2018-06-19',
 '2018-06-26',
 '2018-07-03',
 '2018-07-03',
 '2018-07-10',
 '2018-07-17',
 '2018-07-24',
 '2018-07-24',
 '2018-07-31',
 '2018-08-07',
 '2018-08-14',
 '2018-08-21',
 '2018-08-21',
 '2018-08-21',
 '2018-08-28',
 '2018-09-18',
 '2018-10-02',
 '2018-10-02',
 '2018-10-09',
 '2018-10-09',
 '2018-10-16',
 '2018-11-06',
 '2018-11-06',
 '2018-11-20',
 '2018-11-27',
 '2018-11-27',
 '2018-12-11',
 '2018-12-11',
 '2018-12-18',
 '2018-12-25',
 '2018-12-25',
 '2019-01-01',
 '2019-01-08',
 '2019-01-22',
 '2019-01-29',
 '2019-02-05',
 '2019-02-12',
 '2019-02-12',
 '2019-02-19',
 '2019-02-26',
 '2019-03-26',
 '2019-03-26',
 '2019-04-02',
 '2019-04-02',
 '2019-04-09',
 '2019-04-09',
 '2019-04-09',
 '2019-04-16',
 '2019-04-23',
 '2019-04-23',
 '2019-04-23',
 '2019-04-30',
 '2019-05-07',
 '2019-05-07',
 '2019-05-14',
 '2019-05-14',
 '2019-05-28',
 '2019-05-28',
 '2019-06-04',
 '2019-06-04',
 '2019-06-11',
 '2019-06-18',
 '2019-06-

In [8]:
failed_terms ## at least some errors are 0 results, e.g. Integumentary System Physiological Phenomena

['Male Urogenital Diseases',
 'Natural Science Disciplines',
 'Circulatory and Respiratory Physiological Phenomena',
 'Female Urogenital Diseases and Pregnancy Complications',
 'vitamin C',
 'Reproductive and Urinary Physiological Phenomena',
 'Psychological Phenomena and Processes',
 'Hemic and Lymphatic Diseases',
 'Health Care Facilities, Manpower, and Services',
 'Otorhinolaryngologic Diseases',
 'Stomatognathic Diseases',
 'vitamin D',
 'Digestive System and Oral Physiological Phenomena',
 'paleocortex',
 'Health Care Economics and Organizations',
 'Integumentary System Physiological Phenomena',
 'ancient viruses',
 'ethnic differences',
 'savant',
 'Support of Research',
 'Hormones, Hormone Substitutes, and Hormone Antagonists',
 'Behavioral Disciplines and Activities',
 'Hemic and Immune Systems',
 'substance P',
 'Eukaryota',
 'Musculoskeletal and Neural Physiological Phenomena',
 'Disorders of Environmental Origin',
 'eugenics',
 'Humanities',
 'Health Care Quality, Access, an

## Forma as table and get total counts per term per year

In [9]:
df = pd.concat({k: pd.Series(v) for k, v in res_all.items()})
df = df.reset_index().drop(columns=["level_1"])
df.columns=["term","date"]
df["date"] = pd.to_datetime(df["date"],infer_datetime_formhttp://localhost:8888/notebooks/patentsView_api_req.ipynb#at=True)
df["year"] = df["date"].dt.year
# df["patent_count"] = df.groupby(["term","year"])["date"].transform("size")
df = df.groupby(["term","year"]).size().reset_index().rename(columns={0:"patent_count"})
df

,term,year,patent_count
0,"Amino Acids, Peptides, and Proteins",1976,5
1,"Amino Acids, Peptides, and Proteins",1977,1
2,"Amino Acids, Peptides, and Proteins",1984,1
3,"Amino Acids, Peptides, and Proteins",1986,2
4,"Amino Acids, Peptides, and Proteins",1988,1
...,...,...,...
5809,zebra fish,2012,1
5810,zebra fish,2013,1
5811,zebra fish,2017,1
5812,zebra fish,2018,1


In [11]:
df.nunique()

term            210
year             47
patent_count    516
dtype: int64

In [10]:
if SAVE:
    df.to_csv("patentsview_terms.csv",index=False)